# Set up

In [1]:
from abc import ABC, abstractmethod

# Factory Method Demo

In [2]:
# Abstract Product in Factory Method
# Component in Decorator
class PaymentMethod(ABC):
    @abstractmethod
    def process_payment(self, amount, user):
        pass

In [3]:
# Concrete Products in Factory Method
# Concrete Components in Decorator
class CreditCardPayment(PaymentMethod):
    def process_payment(self, amount, user):
        print(f"Processing credit card payment of {amount}.")

class PayPalPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if not user.is_paypal_verified:
            print("User's PayPal account is not verified.")
            return
        print(f"Processing PayPal payment of {amount}.")

class BankTransferPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if amount <= 3000:
            print("Bank Transfer requires an amount greater than 3000.")
            return
        print(f"Processing bank transfer payment of {amount}.")

class BitCoinPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if amount > 10000:
            print("BitCoin payment cannot process amounts over 10000.")
            return
        print(f"Processing BitCoin payment of {amount}.")

class ApplePayPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if user.region not in ['US', 'UK', 'Canada']:
            print("Apple Pay is not available in your region.")
            return
        print(f"Processing Apple Pay payment of {amount}.")

# # For Expand Case
# class GooglePayPayment(PaymentMethod):
#     def process_payment(self, amount, user):
#         print(f"Processing Google Pay payment of {amount}.")
#         # Add more if you want! 



In [4]:
# Abstract Creator
class PaymentCreator(ABC):
    @abstractmethod
    def create_payment_method(self, method_type):
        pass


In [5]:
# Concreate Creator
class PaymentFactory:
    def create_payment_method(self, method_type):
        if method_type == "credit_card":
            return CreditCardPayment()
        elif method_type == "paypal":
            return PayPalPayment()
        elif method_type == "bank_transfer":
            return BankTransferPayment()
        elif method_type == "bitcoin":
            return BitCoinPayment()
        elif method_type == "apple_pay":
            return ApplePayPayment()
        # # For Expand Case
        # elif method_type == "google_pay":
        #     return GooglePayPayment()
        else:
            raise ValueError("Invalid payment method.")


In [6]:
class User:
    def __init__(self, is_paypal_verified, region):
        self.is_paypal_verified = is_paypal_verified
        self.region = region


In [7]:
if __name__ == "__main__":
    factory = PaymentFactory()
    user = User(is_paypal_verified=True, region='US')
    
    # Pay by Paypal
    payment_method_1 = factory.create_payment_method("paypal")
    payment_method_1.process_payment(5000, user)

    # Pay by Bank Transfer < 3000 (rejected)
    payment_method_2 = factory.create_payment_method("bank_transfer")
    payment_method_2.process_payment(2999, user)

    # Pay by Bank Transfer > 10000 (rejected))
    payment_method_3 = factory.create_payment_method("bitcoin")
    payment_method_3.process_payment(15000, user)

    # Pay by Apple Pay
    payment_method_4 = factory.create_payment_method("apple_pay")
    payment_method_4.process_payment(3000, user)

    # # For Expanded Case
    # payment_method_5 = factory.create_payment_method("google_pay")
    # payment_method_5.process_payment(4000, user)


Processing PayPal payment of 5000.
Bank Transfer requires an amount greater than 3000.
BitCoin payment cannot process amounts over 10000.
Processing Apple Pay payment of 3000.


# Decorator Demo

In [8]:
# Decorator
class PaymentDecorator(PaymentMethod):
    def __init__(self, payment_method):
        self._payment_method = payment_method

    @abstractmethod
    def process_payment(self, amount, user):
        pass


In [9]:
# Concrete Decorators

# ConcreteDecorator - Transaction Fee
class TransactionFeeDecorator(PaymentDecorator):
    def process_payment(self, amount, user):
        fee = amount * 0.02  # 2% transaction fee
        amount_with_fee = amount + fee
        print(f"Adding transaction fee of {fee}. New amount: {amount_with_fee}")
        self._payment_method.process_payment(amount_with_fee, user)

# ConcreteDecorator - Discount
class DiscountDecorator(PaymentDecorator):
    def process_payment(self, amount, user):
        discount = amount * 0.05  # 5% discount
        amount_with_discount = amount - discount
        print(f"Applying discount of {discount}. New amount: {amount_with_discount}")
        self._payment_method.process_payment(amount_with_discount, user)

# ConcreteDecorator - Logging
class LoggingDecorator(PaymentDecorator):
    def process_payment(self, amount, user):
        print(f"Logging payment of {amount} for user {user.id}")
        self._payment_method.process_payment(amount, user)


In [10]:
class User:
    def __init__(self, user_id, is_paypal_verified, region):
        self.id = user_id
        self.is_paypal_verified = is_paypal_verified
        self.region = region


In [11]:
if __name__ == "__main__":
    factory = PaymentFactory()
    user = User(user_id='user123', is_paypal_verified=True, region='US')

    # Tạo phương thức thanh toán cơ bản
    payment_method = factory.create_payment_method("credit_card")

    # Áp dụng Decorators
    payment_with_fee = TransactionFeeDecorator(payment_method)
    payment_with_discount = DiscountDecorator(payment_with_fee)
    payment_with_logging = LoggingDecorator(payment_with_discount)

    # Thực hiện thanh toán với các decorators
    payment_with_logging.process_payment(1000, user)


Logging payment of 1000 for user user123
Applying discount of 50.0. New amount: 950.0
Adding transaction fee of 19.0. New amount: 969.0
Processing credit card payment of 969.0.


# Chain Of Responsibility

In [12]:
# Handler
class PaymentHandler(ABC):
    def __init__(self, next_handler=None):
        self._next_handler = next_handler

    def set_next(self, handler):
        self._next_handler = handler

    @abstractmethod
    def handle(self, amount, user):
        if self._next_handler:
            return self._next_handler.handle(amount, user)
        return True

In [13]:
# Concrete Handlers 
class PayPalVerificationHandler(PaymentHandler):
    def handle(self, amount, user):
        if not user.is_paypal_verified:
            print("User's PayPal account is not verified.")
            return False
        print("User's PayPal account is verified.")
        return super().handle(amount, user)
    
class AmountCheckHandler(PaymentHandler):
    def handle(self, amount, user):
        if amount <= 0:
            print("Invalid payment amount. Must be greater than 0.")
            return False
        if amount > 10000:
            print("Payment amount exceeds the maximum limit of 10000.")
            return False
        print(f"Payment amount {amount} is valid.")
        return super().handle(amount, user)

class FinalPaymentHandler(PaymentHandler):
    def __init__(self, payment_method):
        super().__init__()
        self.payment_method = payment_method

    def handle(self, amount, user):
        print("All checks passed. Processing payment...")
        self.payment_method.process_payment(amount, user)
        return True


In [14]:
class User:
    def __init__(self, user_id, is_paypal_verified, region):
        self.id = user_id
        self.is_paypal_verified = is_paypal_verified
        self.region = region


In [15]:
if __name__ == "__main__":
    # Tạo factory để tạo phương thức thanh toán
    factory = PaymentFactory()
    user = User(user_id='user123', is_paypal_verified=True, region='US')

    # Chọn phương thức thanh toán (CreditCard hoặc PayPal)
    payment_method = factory.create_payment_method("paypal")

    # Tạo các handler cho chuỗi xử lý
    paypal_verification = PayPalVerificationHandler()
    amount_check = AmountCheckHandler()
    final_payment = FinalPaymentHandler(payment_method)

    # Xây dựng chuỗi trách nhiệm
    paypal_verification.set_next(amount_check)
    amount_check.set_next(final_payment)

    # Bắt đầu xử lý thanh toán thông qua chuỗi
    paypal_verification.handle(5000, user)


User's PayPal account is verified.
Payment amount 5000 is valid.
All checks passed. Processing payment...
Processing PayPal payment of 5000.


# Total Combination

In [16]:
from abc import ABC, abstractmethod

# ======= Payment Methods =======
class PaymentMethod(ABC):
    @abstractmethod
    def process_payment(self, amount, user):
        pass

class CreditCardPayment(PaymentMethod):
    def process_payment(self, amount, user):
        print(f"Processing credit card payment of {amount}.")

class PayPalPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if not user.is_paypal_verified:
            print("User's PayPal account is not verified.")
            return
        print(f"Processing PayPal payment of {amount}.")

class BankTransferPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if amount <= 3000:
            print("Bank Transfer requires an amount greater than 3000.")
            return
        print(f"Processing bank transfer payment of {amount}.")

class BitCoinPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if amount > 10000:
            print("BitCoin payment cannot process amounts over 10000.")
            return
        print(f"Processing BitCoin payment of {amount}.")

class ApplePayPayment(PaymentMethod):
    def process_payment(self, amount, user):
        if user.region not in ['US', 'UK', 'Canada']:
            print("Apple Pay is not available in your region.")
            return
        print(f"Processing Apple Pay payment of {amount}.")

# ======= Payment Creation =======
class PaymentFactory:
    def create_payment_method(self, method_type):
        if method_type == "credit_card":
            return CreditCardPayment()
        elif method_type == "paypal":
            return PayPalPayment()
        elif method_type == "bank_transfer":
            return BankTransferPayment()
        elif method_type == "bitcoin":
            return BitCoinPayment()
        elif method_type == "apple_pay":
            return ApplePayPayment()
        else:
            raise ValueError("Invalid payment method.")

# ======= User Class =======
class User:
    def __init__(self, user_id, is_paypal_verified, region):
        self.id = user_id
        self.is_paypal_verified = is_paypal_verified
        self.region = region

# ======= Payment Processing Flow =======
class PaymentHandler(ABC):
    def __init__(self, next_handler=None):
        self._next_handler = next_handler

    def set_next(self, handler):
        self._next_handler = handler

    @abstractmethod
    def handle(self, amount, user):
        if self._next_handler:
            return self._next_handler.handle(amount, user)
        return True

class PayPalVerificationHandler(PaymentHandler):
    def handle(self, amount, user):
        if not user.is_paypal_verified:
            print("User's PayPal account is not verified.")
            return False
        print("User's PayPal account is verified.")
        return super().handle(amount, user)

class AmountCheckHandler(PaymentHandler):
    def handle(self, amount, user):
        if amount <= 0:
            print("Invalid payment amount. Must be greater than 0.")
            return False
        if amount > 10000:
            print("Payment amount exceeds the maximum limit of 10000.")
            return False
        print(f"Payment amount {amount} is valid.")
        return super().handle(amount, user)

class FinalPaymentHandler(PaymentHandler):
    def __init__(self, payment_method):
        super().__init__()
        self.payment_method = payment_method

    def handle(self, amount, user):
        print("All checks passed. Processing payment...")
        self.payment_method.process_payment(amount, user)
        return True

# ======= Payment Enhancements (Optional Decorators) =======
class PaymentEnhancement(PaymentMethod):
    def __init__(self, payment_method):
        self._payment_method = payment_method

class TransactionFee(PaymentEnhancement):
    def process_payment(self, amount, user):
        fee = amount * 0.02
        print(f"Adding transaction fee of {fee}. New amount: {amount + fee}")
        self._payment_method.process_payment(amount + fee, user)

class Discount(PaymentEnhancement):
    def process_payment(self, amount, user):
        discount = amount * 0.05
        print(f"Applying discount of {discount}. New amount: {amount - discount}")
        self._payment_method.process_payment(amount - discount, user)

class PaymentLogger(PaymentEnhancement):
    def process_payment(self, amount, user):
        print(f"Logging payment of {amount} for user {user.id}")
        self._payment_method.process_payment(amount, user)

# ======= Main Payment Process =======
if __name__ == "__main__":
    # Create User
    user = User(user_id='user123', is_paypal_verified=True, region='US')

    # Create Payment Method
    factory = PaymentFactory()
    payment_method = factory.create_payment_method("paypal")

    # Apply Logging, discount and transaction feee
    payment_with_fee = TransactionFee(payment_method)
    payment_with_discount = Discount(payment_with_fee)
    payment_with_logging = PaymentLogger(payment_with_discount)

    # Set up the chain of checks
    paypal_verification = PayPalVerificationHandler()
    amount_check = AmountCheckHandler()
    final_payment = FinalPaymentHandler(payment_with_logging)

    paypal_verification.set_next(amount_check)
    amount_check.set_next(final_payment)

    # Process payment
    paypal_verification.handle(5000, user)


User's PayPal account is verified.
Payment amount 5000 is valid.
All checks passed. Processing payment...
Logging payment of 5000 for user user123
Applying discount of 250.0. New amount: 4750.0
Adding transaction fee of 95.0. New amount: 4845.0
Processing PayPal payment of 4845.0.
